In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [2]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2024-08-07 05:47:24--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.112.4
Connecting to gist.github.com (gist.github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2024-08-07 05:47:24--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv’

registrations.csv   100%[===================>]  14.57K  --.-KB/s    in 0s      

2024-08-07 05:47:24 (60.5 M

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [3]:
# Ваше решение
reg = pd.read_csv('registrations.csv', sep=';')
reg

,user_id,registration_date
0,1,2021-01-01
1,2,2021-01-01
2,3,2021-01-01
3,4,2021-01-01
4,5,2021-01-01
...,...,...
995,996,2021-12-05
996,997,2021-12-05
997,998,2021-12-06
998,999,2021-12-06


In [4]:
ent = pd.read_csv('entries.csv', sep=';')
ent

,user_id,entry_date
0,1,2021-01-01
1,2,2021-01-01
2,3,2021-01-01
3,4,2021-01-01
4,5,2021-01-01
...,...,...
20729,990,2021-12-31
20730,990,2021-12-31
20731,979,2021-12-31
20732,979,2021-12-31


In [5]:
df = reg.merge(ent, how='outer', on='user_id')
df

,user_id,registration_date,entry_date
0,1,2021-01-01,2021-01-01
1,1,2021-01-01,2021-01-03
2,1,2021-01-01,2021-01-04
3,1,2021-01-01,2021-01-04
4,1,2021-01-01,2021-01-05
...,...,...,...
20729,1000,2021-12-07,2021-12-26
20730,1000,2021-12-07,2021-12-28
20731,1000,2021-12-07,2021-12-29
20732,1000,2021-12-07,2021-12-30


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20734 entries, 0 to 20733
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            20734 non-null  int64 
 1   registration_date  20734 non-null  object
 2   entry_date         20734 non-null  object
dtypes: int64(1), object(2)
memory usage: 486.1+ KB


In [7]:
df['registration_date'] = pd.to_datetime(df['registration_date'])
df['entry_date'] = pd.to_datetime(df['entry_date'])

df

,user_id,registration_date,entry_date
0,1,2021-01-01,2021-01-01
1,1,2021-01-01,2021-01-03
2,1,2021-01-01,2021-01-04
3,1,2021-01-01,2021-01-04
4,1,2021-01-01,2021-01-05
...,...,...,...
20729,1000,2021-12-07,2021-12-26
20730,1000,2021-12-07,2021-12-28
20731,1000,2021-12-07,2021-12-29
20732,1000,2021-12-07,2021-12-30


In [8]:
df.isna().any()

user_id              False
registration_date    False
entry_date           False
dtype: bool

In [9]:
df['registration_date'].agg(['min', 'max'])

min   2021-01-01
max   2021-12-07
Name: registration_date, dtype: datetime64[ns]

In [10]:
df_jan = df[df['registration_date'].dt.month==1]

df_jan['user_id'].unique().shape[0]

86

In [11]:
df_jan_15 = df_jan[(df_jan['entry_date'] - df_jan['registration_date']) / np.timedelta64 ( 1 , 'D') == 15]
df_jan_15['user_id'].unique().shape[0]

47

In [12]:
retention_15_day = round(df_jan_15['user_id'].unique().shape[0] / df_jan['user_id'].unique().shape[0] * 100, 5)
retention_15_day

54.65116

In [13]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [14]:
# Ваше решение

df_jan_30 = df_jan[(df_jan['entry_date'] - df_jan['registration_date']) / np.timedelta64 ( 1 , 'D') >= 30]

df_jan_30['user_id'].unique().shape[0]

25

In [15]:
rolling_retention = round(df_jan_30['user_id'].unique().shape[0] / df_jan['user_id'].unique().shape[0] * 100, 5)
rolling_retention

29.06977

In [16]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`

In [ ]:
last_act = df.groupby(by=['user_id', 'registration_date']).max().reset_index()
last_act

,user_id,registration_date,entry_date
0,1,2021-01-01,2021-01-30
1,2,2021-01-01,2021-01-30
2,3,2021-01-01,2021-01-31
3,4,2021-01-01,2021-01-30
4,5,2021-01-01,2021-01-30
...,...,...,...
995,996,2021-12-05,2021-12-31
996,997,2021-12-05,2021-12-31
997,998,2021-12-06,2021-12-30
998,999,2021-12-06,2021-12-31


In [ ]:
last_act['days_visited'] = ((last_act['entry_date'] - last_act['registration_date']) / np.timedelta64 ( 1 , 'D')).astype(int)
last_act

,user_id,registration_date,entry_date,days_visited
0,1,2021-01-01,2021-01-30,29
1,2,2021-01-01,2021-01-30,29
2,3,2021-01-01,2021-01-31,30
3,4,2021-01-01,2021-01-30,29
4,5,2021-01-01,2021-01-30,29
...,...,...,...,...
995,996,2021-12-05,2021-12-31,26
996,997,2021-12-05,2021-12-31,26
997,998,2021-12-06,2021-12-30,24
998,999,2021-12-06,2021-12-31,25


In [ ]:
last_act['is_churned_int'] = last_act['days_visited'].apply(lambda x: 1 if x < 29 else 0)
last_act

,user_id,registration_date,entry_date,days_visited,is_churned_int
0,1,2021-01-01,2021-01-30,29,0
1,2,2021-01-01,2021-01-30,29,0
2,3,2021-01-01,2021-01-31,30,0
3,4,2021-01-01,2021-01-30,29,0
4,5,2021-01-01,2021-01-30,29,0
...,...,...,...,...,...
995,996,2021-12-05,2021-12-31,26,1
996,997,2021-12-05,2021-12-31,26,1
997,998,2021-12-06,2021-12-30,24,1
998,999,2021-12-06,2021-12-31,25,1


In [ ]:
last_act['is_churned_int'].sum()

509

In [ ]:
last_act['user_id'].count()

1000

In [ ]:
churn_29 = last_act['is_churned_int'].sum() / last_act['user_id'].count()
churn_29

0.509

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [ ]:
df_dec_m = df[df['entry_date'] >= '2021-12-01']
df_dec_m

,user_id,registration_date,entry_date
17223,828,2021-11-01,2021-12-01
17260,830,2021-11-01,2021-12-01
17277,831,2021-11-01,2021-12-01
17318,833,2021-11-01,2021-12-01
17360,835,2021-11-01,2021-12-01
...,...,...,...
20729,1000,2021-12-07,2021-12-26
20730,1000,2021-12-07,2021-12-28
20731,1000,2021-12-07,2021-12-29
20732,1000,2021-12-07,2021-12-30


In [ ]:
dec_mau = df_dec_m['user_id'].nunique()
dec_mau

133

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [ ]:
df_dec_w = df[(df['entry_date'] >= '2021-12-25') & (df['entry_date'] <= '2021-12-31')]
df_dec_w

,user_id,registration_date,entry_date
18738,917,2021-12-01,2021-12-27
18739,917,2021-12-01,2021-12-29
18740,917,2021-12-01,2021-12-31
18741,917,2021-12-01,2021-12-31
18758,918,2021-12-01,2021-12-26
...,...,...,...
20729,1000,2021-12-07,2021-12-26
20730,1000,2021-12-07,2021-12-28
20731,1000,2021-12-07,2021-12-29
20732,1000,2021-12-07,2021-12-30


In [ ]:
dec_wau = df_dec_w['user_id'].nunique()

dec_wau

84

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [ ]:
# Ваше решение
df_dec_d = df[df['entry_date'] == '2021-12-31']
df_dec_d



,user_id,registration_date,entry_date
18740,917,2021-12-01,2021-12-31
18741,917,2021-12-01,2021-12-31
18764,918,2021-12-01,2021-12-31
18765,918,2021-12-01,2021-12-31
18861,922,2021-12-01,2021-12-31
...,...,...,...
20565,993,2021-12-04,2021-12-31
20637,996,2021-12-05,2021-12-31
20661,997,2021-12-05,2021-12-31
20708,999,2021-12-06,2021-12-31


In [ ]:
dec_dau = df_dec_d['user_id'].nunique()
dec_dau

47

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [ ]:
df['entry_date'].agg(['min', 'max'])


min   2021-01-01
max   2021-12-31
Name: entry_date, dtype: datetime64[ns]

In [ ]:
df_avg_m = df.groupby(pd.PeriodIndex(df['entry_date'], freq="M"))['user_id'].nunique().reset_index()
df_avg_m

,entry_date,user_id
0,2021-01,86
1,2021-02,75
2,2021-03,162
3,2021-04,75
4,2021-05,119
5,2021-06,65
6,2021-07,111
7,2021-08,83
8,2021-09,77
9,2021-10,153


In [ ]:
avg_mau = round(df_avg_m['user_id'].mean(), 5)
avg_mau

102.58333

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [ ]:
df_avg_w = df.groupby(pd.PeriodIndex(df['entry_date'], freq="W"))['user_id'].nunique().reset_index()
df_avg_w

,entry_date,user_id
0,2020-12-28/2021-01-03,78
1,2021-01-04/2021-01-10,86
2,2021-01-11/2021-01-17,86
3,2021-01-18/2021-01-24,86
4,2021-01-25/2021-01-31,86
5,2021-02-01/2021-02-07,54
6,2021-02-08/2021-02-14,60
7,2021-02-15/2021-02-21,60
8,2021-02-22/2021-02-28,54
9,2021-03-01/2021-03-07,162


In [ ]:
avg_wau = round(df_avg_w['user_id'].mean(), 5)
avg_wau


89.86792

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [ ]:
df_avg_d = df.groupby(pd.PeriodIndex(df['entry_date'], freq="D"))['user_id'].nunique().reset_index()
df_avg_d

,entry_date,user_id
0,2021-01-01,54
1,2021-01-02,46
2,2021-01-03,47
3,2021-01-04,47
4,2021-01-05,53
...,...,...
360,2021-12-27,52
361,2021-12-28,44
362,2021-12-29,38
363,2021-12-30,46


In [ ]:
avg_dau = round(df_avg_d['user_id'].mean(), 5)
avg_dau


40.5589

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
